# Install packages

In [10]:
!apt install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pandas numpy transformers[torch] huggingface_hub tiktoken einops pytest flash-attn ipywidgets jinja2==3.1.0 

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached transformers-4.48.1-py3-none-any.whl (9.7 MB)
  Using cached huggingface_hub-0.27.1-py3-none-any.whl (450 kB)
  Using cached tiktoken-0.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached einops-0.8.0-py3-none-any.whl (43 kB)
  Using cached pytest-8.3.4-py3-none-any.whl (343 kB)
  Using cached flash_attn-2.7.3.tar.gz (3.2 MB)
  Preparing metadata (setup.py) ... one
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 KB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 KB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 44.6 M

In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 51.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 14.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 KB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 16.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 KB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 12.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 44.1 MB/s eta 0:00:0

# Import packages

In [1]:
from pyspark.sql import SparkSession
import json
from pyspark.sql.functions import col, desc
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [2]:
hotel_reviews_path = './input/review.txt'
hotel_list_path = './input/offering.txt'

In [3]:
hotel_list_json_data = []
with open(hotel_list_path) as f:
    for line in f:
        hotel_list_json_data.append(json.loads(line.strip()))
hotel_list_json_data

[{'hotel_class': 4.0,
  'region_id': 60763,
  'url': 'http://www.tripadvisor.com/Hotel_Review-g60763-d113317-Reviews-Casablanca_Hotel_Times_Square-New_York_City_New_York.html',
  'phone': '',
  'details': None,
  'address': {'region': 'NY',
   'street-address': '147 West 43rd Street',
   'postal-code': '10036',
   'locality': 'New York City'},
  'type': 'hotel',
  'id': 113317,
  'name': 'Casablanca Hotel Times Square'},
 {'hotel_class': 5.0,
  'region_id': 32655,
  'url': 'http://www.tripadvisor.com/Hotel_Review-g32655-d76049-Reviews-Four_Seasons_Hotel_Los_Angeles_at_Beverly_Hills-Los_Angeles_California.html',
  'phone': '',
  'details': None,
  'address': {'region': 'CA',
   'street-address': '300 S Doheny Dr',
   'postal-code': '90048',
   'locality': 'Los Angeles'},
  'type': 'hotel',
  'id': 76049,
  'name': 'Four Seasons Hotel Los Angeles at Beverly Hills'},
 {'hotel_class': 3.5,
  'region_id': 60763,
  'url': 'http://www.tripadvisor.com/Hotel_Review-g60763-d99352-Reviews-Hilton_

In [4]:
def keys_subkeys(data):
    keys = []
    subkeys = {}
    for x in data:
        keys.extend(x.keys())
        for key in x.keys():
            if isinstance(x[key], dict):
                if key not in subkeys:
                    subkeys[key] = []
                subkeys[key].extend(x[key].keys())

    keys = list(set(keys))
    for key in subkeys.keys():
        subkeys[key] = list(set(subkeys[key]))
    return keys, subkeys

In [5]:
keys, subkeys = keys_subkeys(hotel_list_json_data)

In [6]:
print(keys)
print(subkeys)

['address', 'phone', 'type', 'url', 'hotel_class', 'id', 'name', 'region_id', 'details']
{'address': ['street-address', 'postal-code', 'region', 'locality']}


In [7]:
for i in hotel_list_json_data:
    if isinstance(i['address'], dict):
        address = ''
        if 'street-address' in i['address'].keys():
            address = i['address']['street-address']

        if 'locality' in i['address'].keys():
            address = address + ', '+i['address']['locality']

        if 'region' in i['address'].keys():
            address = address + ', '+i['address']['region']

        if 'postal-code' in i['address'].keys():
            address = address +' '+i['address']['postal-code']
        i['address'] = address

In [8]:
hotel_reviews_json_data = []
with open(hotel_reviews_path, encoding='cp1252', errors='replace') as f:
    for line in f:
        try:
            # Replace unescaped newline characters within the line
            cleaned_line = line.strip().replace('\n', '\\n')
            # Attempt to parse the JSON line
            hotel_reviews_json_data.append(json.loads(cleaned_line))
        except json.JSONDecodeError as e:
            print(f"Failed to decode JSON line: {e}")
            print(f"Line causing error: {line}")

In [9]:
keys, subkeys = keys_subkeys(hotel_reviews_json_data)

In [10]:
print(keys)
print(subkeys)

['text', 'date', 'num_helpful_votes', 'id', 'ratings', 'title', 'offering_id', 'date_stayed', 'author', 'via_mobile']
{'ratings': ['rooms', 'check_in_front_desk', 'cleanliness', 'location', 'business_service_(e_g_internet_access)', 'overall', 'value', 'sleep_quality', 'service'], 'author': ['num_cities', 'num_type_reviews', 'username', 'location', 'num_helpful_votes', 'id', 'num_reviews']}


In [11]:
for i in hotel_reviews_json_data:
    if isinstance(i['ratings'], dict):
        if 'business_service_(e_g_internet_access)' in i['ratings'].keys():
            i['business_service'] = i['ratings']['business_service_(e_g_internet_access)']
        else:
            i['business_service'] = None

        if 'rooms' in i['ratings'].keys():
            i['rooms'] = i['ratings']['rooms']
        else:
            i['rooms'] = None

        if 'value' in i['ratings'].keys():
            i['value'] = i['ratings']['value']
        else:
            i['value'] = None

        if 'sleep_quality' in i['ratings'].keys():
            i['sleep_quality'] = i['ratings']['sleep_quality']
        else:
            i['sleep_quality'] = None

        if 'overall' in i['ratings'].keys():
            i['overall'] = i['ratings']['overall']
        else:
            i['overall'] = None

        if 'check_in_front_desk' in i['ratings'].keys():
            i['front_desk'] = i['ratings']['check_in_front_desk']
        else:
            i['front_desk'] = None

        if 'cleanliness' in i['ratings'].keys():
            i['cleanliness'] = i['ratings']['cleanliness']
        else:
            i['cleanliness'] = None

        if 'location' in i['ratings'].keys():
            i['location'] = i['ratings']['location']
        else:
            i['location'] = None

        if 'service' in i['ratings'].keys():
            i['service'] = i['ratings']['service']
        else:
            i['service'] = None

        del(i['ratings'])

        i['author'] = i['author']['id']

In [12]:
len(hotel_reviews_json_data)

878561

In [13]:
keys, subkeys = keys_subkeys(hotel_reviews_json_data)

In [14]:
print(keys)
print(subkeys)

['text', 'business_service', 'rooms', 'front_desk', 'date', 'cleanliness', 'location', 'overall', 'service', 'num_helpful_votes', 'id', 'title', 'offering_id', 'date_stayed', 'sleep_quality', 'author', 'value', 'via_mobile']
{}


In [15]:
hotel_reviews_json_data[-2]

{'title': '“Gerne wieder”',
 'text': 'Sehr schön ausgestattetes Hotel in bester Lage zur Innenstadt/Duupont Circle/Georgetown. Sehr gute Anbindung an öffentliche Verkehrsmittel und nahe an guten Restaurants. Aufmerksamer Service und innovatives Management (jeden Tag ein Empfang mit Wein für Gäste des Hotels in der Lobby am späten Nachmittag; morgens Tee und Kaffee/Tageszeitung gratis)',
 'author': '',
 'date_stayed': 'July 2008',
 'offering_id': 84093,
 'num_helpful_votes': 0,
 'date': 'April 1, 2008',
 'id': 15564508,
 'via_mobile': False,
 'business_service': None,
 'rooms': 5.0,
 'value': None,
 'sleep_quality': None,
 'overall': 5.0,
 'front_desk': None,
 'cleanliness': 5.0,
 'location': None,
 'service': 5.0}

In [16]:
cleaned_hotel_reviews_path = './cleaned/hotel_reviews.json'
cleaned_hotel_list_path = './cleaned/hotel_list.json'

In [17]:
with open(cleaned_hotel_reviews_path, 'w') as f:
    for i in hotel_reviews_json_data:
        json.dump(i, f)
        f.write('\n')

In [18]:
with open(cleaned_hotel_list_path, 'w') as f:
    for i in hotel_list_json_data:
        json.dump(i, f)
        f.write('\n')

In [19]:
spark = SparkSession.builder.appName("hotel_reviews").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/21 22:52:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
hotel_reviews_df = spark.read.option('inferSchema', 'true').json(cleaned_hotel_reviews_path)

hotel_reviews_df.show(5)

+--------------------+----------------+-----------+-----------------+-------------+----------+---------+--------+-----------------+-----------+-------+-----+-------+-------------+--------------------+--------------------+-----+----------+
|              author|business_service|cleanliness|             date|  date_stayed|front_desk|       id|location|num_helpful_votes|offering_id|overall|rooms|service|sleep_quality|                text|               title|value|via_mobile|
+--------------------+----------------+-----------+-----------------+-------------+----------+---------+--------+-----------------+-----------+-------+-----+-------+-------------+--------------------+--------------------+-----+----------+
|8C0B42FF3C0FA366A...|            NULL|        5.0|December 17, 2012|December 2012|      NULL|147643103|     5.0|                0|      93338|    5.0|  5.0|    5.0|          5.0|Stayed in a king ...|“Truly is "Jewel ...|  5.0|     false|
|E3C85CA9DBBBC77E0...|            NULL|     

In [21]:
hotel_reviews_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- business_service: double (nullable = true)
 |-- cleanliness: double (nullable = true)
 |-- date: string (nullable = true)
 |-- date_stayed: string (nullable = true)
 |-- front_desk: double (nullable = true)
 |-- id: long (nullable = true)
 |-- location: double (nullable = true)
 |-- num_helpful_votes: long (nullable = true)
 |-- offering_id: long (nullable = true)
 |-- overall: double (nullable = true)
 |-- rooms: double (nullable = true)
 |-- service: double (nullable = true)
 |-- sleep_quality: double (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- value: double (nullable = true)
 |-- via_mobile: boolean (nullable = true)



In [22]:
hotel_reviews_df.count()

878561

In [23]:
hotel_reviews_df.describe()

DataFrame[summary: string, author: string, business_service: string, cleanliness: string, date: string, date_stayed: string, front_desk: string, id: string, location: string, num_helpful_votes: string, offering_id: string, overall: string, rooms: string, service: string, sleep_quality: string, text: string, title: string, value: string]

In [24]:
hotel_list_df = spark.read.option('inferSchema', 'true').json(cleaned_hotel_list_path)
hotel_list_df.show(5)

+--------------------+-------+-----------+------+--------------------+-----+---------+-----+--------------------+
|             address|details|hotel_class|    id|                name|phone|region_id| type|                 url|
+--------------------+-------+-----------+------+--------------------+-----+---------+-----+--------------------+
|147 West 43rd Str...|   NULL|        4.0|113317|Casablanca Hotel ...|     |    60763|hotel|http://www.tripad...|
|300 S Doheny Dr, ...|   NULL|        5.0| 76049|Four Seasons Hote...|     |    32655|hotel|http://www.tripad...|
|790 Eighth Avenue...|   NULL|        3.5| 99352|Hilton Garden Inn...|     |    60763|hotel|http://www.tripad...|
|152 West 51st Str...|   NULL|        4.0| 93589|The Michelangelo ...|     |    60763|hotel|http://www.tripad...|
|130 West 46th Str...|   NULL|        4.0|217616|The Muse Hotel Ne...|     |    60763|hotel|http://www.tripad...|
+--------------------+-------+-----------+------+--------------------+-----+---------+--

In [25]:
hotel_list_df.count()

4333

In [26]:
hotel_list_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- details: string (nullable = true)
 |-- hotel_class: double (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- region_id: long (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)



In [27]:
hotel_list_df.createOrReplaceTempView("hotels")
hotel_reviews_df.createOrReplaceTempView("hotel_reviews")

In [28]:
spark.sql("SELECT * FROM hotels").show(truncate=False)

+------------------------------------------------------------------+-------+-----------+-------+-----------------------------------------------+-----+---------+-----+------------------------------------------------------------------------------------------------------------------------------------------+
|address                                                           |details|hotel_class|id     |name                                           |phone|region_id|type |url                                                                                                                                       |
+------------------------------------------------------------------+-------+-----------+-------+-----------------------------------------------+-----+---------+-----+------------------------------------------------------------------------------------------------------------------------------------------+
|147 West 43rd Street, New York City, NY 10036                     |NULL   |4.0   

In [29]:
spark.sql("SELECT count(*) FROM hotel_reviews where service is NULL").show(truncate=False)

[Stage 11:=============================================>            (7 + 2) / 9]

+--------+
|count(1)|
+--------+
|117643  |
+--------+



In [30]:
spark.sql("SELECT hotels.id, hotels.name, avg(hotel_reviews.overall) as avg_overall_rating, count(hotel_reviews.author) as author_count FROM hotels join hotel_reviews on hotels.id=hotel_reviews.offering_id group by hotels.id, hotels.name order by avg(hotel_reviews.overall) desc").createOrReplaceTempView("avg_overall_rating")

In [31]:
top_negatively_rated_hotels = spark.sql("""
    SELECT id
    FROM avg_overall_rating
    ORDER BY avg_overall_rating ASC, author_count DESC
    LIMIT 5
""")
top_negatively_rated_hotels_ids = [row.id for row in top_negatively_rated_hotels.collect()]
top_negatively_rated_hotels_ids

[122337, 643318, 83086, 73985, 589144]

In [32]:
top_negatively_rated_reviews = spark.table("hotel_reviews").filter(col("offering_id").isin(top_negatively_rated_hotels_ids)).orderBy("offering_id", "overall", desc("num_helpful_votes"))

In [33]:
top_negatively_rated_reviews.show()

[Stage 18:=============================================>            (7 + 2) / 9]

+--------------------+----------------+-----------+------------------+--------------+----------+---------+--------+-----------------+-----------+-------+-----+-------+-------------+--------------------+--------------------+-----+----------+
|              author|business_service|cleanliness|              date|   date_stayed|front_desk|       id|location|num_helpful_votes|offering_id|overall|rooms|service|sleep_quality|                text|               title|value|via_mobile|
+--------------------+----------------+-----------+------------------+--------------+----------+---------+--------+-----------------+-----------+-------+-----+-------+-------------+--------------------+--------------------+-----+----------+
|81EF4288C8B883788...|            NULL|        1.0|   January 7, 2012|  January 2012|      NULL|122729551|     1.0|                4|      73985|    1.0|  1.0|    3.0|          1.0|DO NOT STAY HERE....|   “Hotel from Hell”|  1.0|     false|
|2FDCE9A9786264CE7...|            NU

In [34]:
# Create groups based on offering_id and limit each group to 5 reviews sorted by num_helpful_votes in descending order
limited_reviews = top_negatively_rated_reviews.toPandas().groupby("offering_id").apply(
    lambda x: x.sort_values("num_helpful_votes", ascending=False).head(5)
).reset_index(drop=True)

/tmp/ipykernel_143701/2252334871.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  limited_reviews = top_negatively_rated_reviews.toPandas().groupby("offering_id").apply(


In [35]:
limited_reviews['concatenated'] = limited_reviews['title'] + " " + limited_reviews['text']
limited_reviews = limited_reviews[['offering_id', 'concatenated']]
limited_reviews

,offering_id,concatenated
0,73985,“Hotel from Hell” DO NOT STAY HERE. The rooms ...
1,73985,“Scary” This is probably the scariest motel I ...
2,73985,“Pay the 10 dollars more for somewhere else” T...
3,73985,“Bed Bugs!!!!!!!!” I am at home in MN covered ...
4,73985,“PIOR HOTEL DO MUNDO!!!” Se você dá o mínimo d...
5,83086,“BED BUGS AND NASTY THINGS” i stayed at the ro...
6,83086,“There is a bullet hole in our refrigerator!” ...
7,83086,"“A Smelly, Grimy Horror” My friend and I rolle..."
8,83086,“ESPN and HBO and a number of other ammenities...
9,83086,“BED BUGS IN EVERY ROOM” I was laying down and...


In [36]:
grouped_reviews = limited_reviews.groupby("offering_id")

In [37]:
for offering_id, group in grouped_reviews:
    print(f"Offering ID: {offering_id}")
    print(group)
    print("\n")

Offering ID: 73985
   offering_id                                       concatenated
0        73985  “Hotel from Hell” DO NOT STAY HERE. The rooms ...
1        73985  “Scary” This is probably the scariest motel I ...
2        73985  “Pay the 10 dollars more for somewhere else” T...
3        73985  “Bed Bugs!!!!!!!!” I am at home in MN covered ...
4        73985  “PIOR HOTEL DO MUNDO!!!” Se você dá o mínimo d...


Offering ID: 83086
   offering_id                                       concatenated
5        83086  “BED BUGS AND NASTY THINGS” i stayed at the ro...
6        83086  “There is a bullet hole in our refrigerator!” ...
7        83086  “A Smelly, Grimy Horror” My friend and I rolle...
8        83086  “ESPN and HBO and a number of other ammenities...
9        83086  “BED BUGS IN EVERY ROOM” I was laying down and...


Offering ID: 122337
    offering_id                                       concatenated
10       122337  “Parker Spruce” Don't be fooled by the weekly ...
11       122

In [38]:
# Create a dictionary where each key is the offering_id and the value is the concatenated reviews
reviews_dict = grouped_reviews['concatenated'].apply(lambda x: ' '.join(x)).to_dict()

# Model code

## Results with gemma-2-9b

In [40]:
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-9b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)


def get_pain_points(reviews):
    input_text = f"Based on the following hotel reviews, please identify and describe the major pain points and issues guests experienced. Suggest how these issues could be improved in the hotel:\n\n{reviews}\n\n"
    messages = [{"role": "user", "content": input_text}]
    output = pipe(messages, max_new_tokens=1000)
    return output[0]['generated_text'][-1]

# Analyze reviews for each hotel
pain_points = {}
for offering_id, reviews in reviews_dict.items():
    print("--------------------------------------------------------------------------------------")
    pain_points[offering_id] = get_pain_points(reviews)["content"]
    print(f"Hotel ID: {offering_id}\nMajor Pain Points and Suggestions for Improvement (Llama-3) for hotel ID {offering_id}:\n{pain_points[offering_id]}\n")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


--------------------------------------------------------------------------------------
Hotel ID: 73985
Major Pain Points and Suggestions for Improvement (Llama-3) for hotel ID 73985:
## Major Pain Points and Issues at the Hotel:

Based on the reviews, the hotel suffers from severe issues across multiple categories:

**1. Hygiene and Cleanliness:**

* **Dirty rooms:**  Guests report finding dead bugs, stains on bedding, hair in the bed, mold in the bathrooms, and general dust and grime.
* **Uncleanliness:**  The reviews mention a strong smell of cigarettes, suggesting poor ventilation and cleaning practices.

**2. Safety and Security:**

* **Broken locks:**  Multiple guests mention malfunctioning locks on doors, raising concerns about security.
* **Poor lighting and location:**  The hotel's location is described as difficult to find and poorly lit, creating a sense of vulnerability.
* **Drug activity and prostitution:**  One guest explicitly states witnessing drug activity and prostitut

## Results with Phi-3-small-8k

In [40]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-small-8k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-small-8k-instruct",   
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 1000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

# # Define a function to get pain points from reviews
def get_pain_points(reviews):
    input_text = f"Based on the following hotel reviews, please identify and describe the major pain points and issues guests experienced. Suggest how these issues could be improved in the hotel:\n\n{reviews}\n\n"
    messages = [{"role": "user", "content": input_text}]
    input_ids = tokenizer(input_text, return_tensors="pt")
    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

# Analyze reviews for each hotel
pain_points = {}
for offering_id, reviews in reviews_dict.items():
    print("--------------------------------------------------------------------------------------")
    pain_points[offering_id] = get_pain_points(reviews)
    print(f"Hotel ID: {offering_id}\nMajor Pain Points and Suggestions for Improvement (Phi-3) for hotel ID {offering_id}:\n{pain_points[offering_id]}\n")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


--------------------------------------------------------------------------------------


/home/ubuntu/.cache/huggingface/modules/transformers_modules/microsoft/Phi-3-small-8k-instruct/1535ae26fb4faada95c6950e8bc6e867cdad6b00/triton_flash_blocksparse_attn.py:88: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  x = [xi.to_sparse_csr() for xi in x]


Hotel ID: 73985
Major Pain Points and Suggestions for Improvement (Phi-3) for hotel ID 73985:
Major pain points and issues experienced by guests:

1. Poor room conditions: Guests reported dirty, stained, and old furniture, as well as dead bugs on the floor. The rooms were also poorly lit and had a strong smell of smoke and mold.

2. Safety concerns: Guests felt unsafe due to the presence of suspicious individuals, poor lighting, and inadequate security measures such as a large gap under the door and a non-functional bolt lock.

3. Lack of cleanliness: Guests found hair in the bed, moldy bathrooms, and broken light bulbs. The overall cleanliness of the hotel was poor, with dust on the furniture and trash in the bins.

4. Bed bugs: One guest reported being bitten by bed bugs, which is a significant health and safety concern.

5. Inadequate customer service: Guests were dissatisfied with the hotel's management, as they did not receive assistance or a satisfactory response to their complai

## Results with Meta-Llama-3-8B

In [42]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

def get_pain_points(reviews):
    input_text = f"Based on the following hotel reviews, please identify and describe the major pain points and issues guests experienced. Suggest how these issues could be improved in the hotel:\n\n{reviews}\n\n"
    messages = [{"role": "user", "content": input_text}]
    terminators = [pipe.tokenizer.eos_token_id, pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    output = pipe(messages, max_new_tokens=1000, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
    return output[0]['generated_text'][-1]

# Analyze reviews for each hotel
pain_points = {}
for offering_id, reviews in reviews_dict.items():
    print("--------------------------------------------------------------------------------------")
    pain_points[offering_id] = get_pain_points(reviews)["content"]
    print(f"Hotel ID: {offering_id}\nMajor Pain Points and Suggestions for Improvement (Llama-3) for hotel ID {offering_id}:\n{pain_points[offering_id]}\n")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


--------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hotel ID: 73985
Major Pain Points and Suggestions for Improvement (Llama-3) for hotel ID 73985:
Based on the hotel reviews, the major pain points and issues guests experienced are:

1. Poor room condition:
	* Rooms were in terrible shape, with old and broken furniture, stained beds, and moldy bathrooms.
	* Guests reported finding dead bugs, hair, and stains on the bed and carpets.
2. Unhygienic environment:
	* The hotel had a strong smell of old cigarettes, and guests reported finding trash and filth in the rooms and common areas.
3. Safety concerns:
	* Guests reported hearing strange noises and seeing suspicious activity, including prostitution and drug use.
	* The hotel's security was lacking, with no cameras or gates to secure parking areas.
4. Uncomfortable and unclean bedding:
	* Guests reported finding hair and stains on the bed, and some even contracted bed bugs.
5. Poor lighting and navigation:
	* Guests struggled to find the hotel due to poor signage and lighting in the area.


...

Hotel ID: 83086
Major Pain Points and Suggestions for Improvement (Llama-3) for hotel ID 83086:
Based on the hotel reviews, the major pain points and issues guests experienced at the Royal Palace hotel are:

1. **Bed Bugs**: Guests reported finding bed bugs in their rooms, with some even having bites and allergic reactions. The issue was persistent, with some guests experiencing it in multiple rooms.
2. **Filth and Disrepair**: Guests described the hotel as filthy, with dirty carpets, grimy walls, and broken furniture. The hotel's condition was so poor that some guests mentioned it was like a "wreck" or a "sewer rat" would be a better place to stay.
3. **Limited Channel Selection**: The hotel only had a few channels, with some guests complaining that the owners would change the channel at will.
4. **Poor Customer Service**: Guests reported rude and aggressive behavior from the hotel owners, including one guest being spat on by the owner.
5. **Safety Concerns**: Some guests mentioned sa

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hotel ID: 122337
Major Pain Points and Suggestions for Improvement (Llama-3) for hotel ID 122337:
Based on the reviews, the major pain points and issues guests experienced at the Parker Spruce hotel are:

1. Uncooperative and rude staff
2. Unhygienic and dirty conditions, including mice in the rooms and stained sheets
3. Poor maintenance and outdated furniture and fixtures
4. Unpleasant and overwhelming atmosphere, including a strong stench and seedy clientele
5. Lack of cleanliness and maintenance in the bathrooms
6. Unsecured doors and potential safety risks
7. Overpriced rates, especially considering the poor quality of the hotel
8. Unwelcome and unsavory clientele, including prostitutes, pimps, and drug users

To improve these issues, the hotel could consider the following:

1. Staff training: Provide training to staff on customer service, conflict resolution, and communication skills to improve their interactions with guests.
2. Deep cleaning and sanitizing: Conduct a thorough dee

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hotel ID: 589144
Major Pain Points and Suggestions for Improvement (Llama-3) for hotel ID 589144:
Based on the hotel reviews, the major pain points and issues guests experienced are:

1. **Dirty and unhygienic conditions**: Guests reported finding cigarette burns, stained carpets, and dirty furniture in their rooms. The bathrooms were also described as gross, with mold, missing tiles, and patchy repairs.
2. **Poor maintenance**: The hotel's sliding glass doors, walls, and floors were described as thin and flimsy, allowing constant car noise and outside sounds to penetrate the rooms.
3. **Unpleasant odors**: Guests complained about strong smells of stale cigarette smoke, old sweat, and cigarrette butts.
4. **Rude and unhelpful staff**: Several guests reported encountering rude and unhelpful staff, including the front desk clerk who was on a smoke break and didn't seem to care about their concerns.
5. **Lack of amenities**: The hotel lacked basic amenities, such as a continental breakfas